<a href="https://colab.research.google.com/github/Shushruth105/Real-and-fake-face-detection/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q matplotlib pillow

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageChops
import os, io
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from google.colab import files
import zipfile

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ciplab/real-and-fake-face-detection
with zipfile.ZipFile("real-and-fake-face-detection.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

Saving archive.zip to archive.zip
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


FileNotFoundError: [Errno 2] No such file or directory: 'real-and-fake-face-detection.zip'

In [6]:
def get_ela_image(path, quality=90, scale=300):
    original = Image.open(path).convert('RGB')
    buffer = io.BytesIO()
    original.save(buffer, 'JPEG', quality=quality)
    buffer.seek(0)
    compressed = Image.open(buffer)
    ela_image = ImageChops.difference(original, compressed)
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema]) or 1
    ela_image = ela_image.point(lambda x: min(255, x * (scale / max_diff)))
    return ela_image

In [7]:
real_dir = "data/real_and_fake_face/training_real/"
fake_dir = "data/real_and_fake_face/training_fake/"
image_size = (128, 128)
X, y = [], []

def load_images(folder, label):
    for file in os.listdir(folder):
        try:
            path = os.path.join(folder, file)
            ela = get_ela_image(path).resize(image_size)
            X.append(np.array(ela) / 255.0)
            y.append(label)
        except:
            continue

load_images(real_dir, 0)
load_images(fake_dir, 1)

X = np.array(X)
y = np.array(y)


FileNotFoundError: [Errno 2] No such file or directory: 'data/real_and_fake_face/training_real/'